## Step 1: Mounting Google Drive and Installing Dependencies

In [ ]:
# Mount Google Drive
from google.colab import drive, files
drive.mount('/content/drive')

# Navigate to the repo folder
%cd /content/drive/MyDrive/llm-finetuning-project/llm-finetuning-summarizer

# List repo contents
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/llm-finetuning-project/llm-finetuning-summarizer
data	    LICENSE  notebooks	      qa_pairs	 results  wandb
deployment  models   project_plan.md  README.md  scripts


## Step 2: Installing Dependencies and Importing Libraries

In [ ]:
!pip install bitsandbytes --prefer-binary --extra-index-url https://download.pytorch.org/whl/cu118 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 14.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers accelerate peft datasets wandb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
import torch
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    pipeline,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    TrainingArguments,
    EvalPrediction,
    Trainer,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_from_disk
from huggingface_hub import login, HfApi, HfFolder, notebook_login
import wandb
import math
from typing import Dict
import matplotlib.pyplot as plt

In [ ]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: samyakshrestha (samyakshrestha-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
login()

## Step 3: Loading Tokenized Dataset

In [ ]:
data_path = "./data/tokenized_dataset"

In [ ]:
dataset = load_from_disk(data_path)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 210
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 24
    })
})


## Step 4: Verifying GPU and Environment

In [ ]:
# Check for GPU availability and set device
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    device = torch.device("cuda")
    print(f"GPU detected: {device_name}")
else:
    device = torch.device("cpu")
    print("GPU not detected. Using CPU instead.")

print(f"Running on device: {device}")

GPU detected: NVIDIA A100-SXM4-40GB
Running on device: cuda


## Step 5: Configuring LoRA  for PEFT (Parameter-Efficient Fine-Tuning)

In this step, we define the configuration for **LoRA (Low-Rank Adaptation)** using the `LoraConfig` class from the `peft` library. LoRA is a widely adopted technique in fine-tuning large language models efficiently by injecting trainable low-rank matrices into specific parts of the transformer architecture.

Key configuration parameters:
- `r=8`: The rank of the LoRA update matrices. A commonly used value that balances adaptation capacity with compute efficiency.
- `lora_alpha=16`: Scaling factor that helps stabilize training. The effective weight update is scaled by `alpha / r`.
- `target_modules=["q_proj", "v_proj"]`: These are the projection layers in the self-attention mechanism where LoRA is applied.
- `bias="none"`: We do not adapt any bias terms to preserve memory efficiency and avoid unintended interactions.
- `lora_dropout=0.05`: Introduces mild regularization to prevent overfitting in low-data regimes.
- `task_type="CAUSAL_LM"`: Indicates that we are fine-tuning a causal language model (e.g., Mistral).

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,        # This is a text generation task
    r=8,                                 # Rank of the low-rank matrices
    lora_alpha=16,                       # Scaling factor
    lora_dropout=0.05,                   # Dropout to prevent overfitting
    bias="none",                         # Do not update bias terms
    target_modules=["q_proj", "v_proj"]  # Modules to apply LoRA on (common in transformers)
)

## Step 6: Initialzing Model with qLoRA and PEFT

In this section, we prepare our base model for parameter-efficient fine-tuning using the **qLoRA** framework. This allows us to fine-tune large language models (LLMs) in resource-constrained environments by combining two powerful strategies:

1. **Quantized Loading (qLoRA)**  
   Using the `BitsAndBytesConfig`, we load the base model in **4-bit precision** via the `bitsandbytes` library. This drastically reduces memory usage while preserving performance. We specify:
   - `load_in_4bit=True`: Activates 4-bit quantization.
   - `bnb_4bit_compute_dtype=torch.float16`: Uses half-precision during computation for faster GPU execution.
   - `bnb_4bit_use_double_quant=True`: Applies an extra layer of quantization to improve compression.
   - `bnb_4bit_quant_type="nf4"`: Uses NormalFloat-4 (NF4), an information-theoretic quantization format optimized for language models.

2. **LoRA Adapter Injection (PEFT)**  
   We then apply `get_peft_model()` to the quantized base model with a custom `LoraConfig`, which:
   - Inserts low-rank matrices into selected attention layers (here: `q_proj` and `v_proj`)
   - Trains only a small number of parameters (∼0.05%), freezing the rest
   - Supports causal language modeling (`task_type="CAUSAL_LM"`), suitable for instruction-following tasks

> **Why this matters**: This step transforms a massive pretrained model into a lightweight, fine-tunable system without sacrificing generalization. The result is a **LoRA-injected, quantization-aware LLM** ready for efficient training on our domain-specific QA data.

In [ ]:
base_model_name = "mistralai/Mistral-7B-Instruct-v0.3"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [ ]:
# Load the model with quantization
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"  # Automatically puts layers on the GPU if available
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Prepare the model for k-bit training (important!)
base_model = prepare_model_for_kbit_training(base_model)

In [ ]:
# Inject LoRA adapters into the base model
model = get_peft_model(base_model, peft_config)

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
model.config.use_cache = False

In [ ]:
# Send the model to the correct device
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj)

In [ ]:
# Print summary
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 7,251,431,424 || trainable%: 0.0470


## Step 7: Defining TrainingArguments for Supervised Fine-Tuning

Here, we configure the `TrainingArguments` object, which tells the Hugging Face `Trainer` how to perform training. These arguments specify:

- The output directory for saving model checkpoints and logs
- Batch size per device and gradient accumulation to simulate larger batches
- Total number of epochs (passes over the dataset)
- Evaluation and logging frequency
- Learning rate and scheduler strategy
- Mixed precision training (implicitly enabled by bitsandbytes)

These are tuned for training with a small dataset using LoRA adapters on a quantized (4-bit) model. All training will be tracked under the `./results/finetuned-mistral` folder.

In [ ]:
output_dir = "./results/finetuned-mistral"

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=6,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    logging_first_step=True,
    report_to="wandb",
    run_name="finetune-mistral-lora",
    bf16=True
)

## Step 8: Adding Perplexity as an Evaluation Metric

To complement training and validation loss, we log **perplexity**, a widely used metric in language modeling that intuitively reflects how "surprised" the model is by the correct output. Lower perplexity indicates better predictive performance.

This step defines a `compute_metrics` function, which is passed to the `Trainer` to automatically compute and log perplexity at each evaluation step.

Key points:
- We use the model's logits and ground truth labels to compute cross-entropy loss.
- From the loss, we derive **perplexity** as `exp(loss)`.
- This does **not affect training**, and is only run during evaluation.
- The results are logged both in the notebook and in **Weights & Biases (wandb)** for real-time visualization.

This is a standard best practice in instruction fine-tuning, and is fully compatible with LoRA + 4-bit quantization.

In [ ]:
def compute_metrics(eval_pred: EvalPrediction) -> Dict:
    logits, labels = eval_pred

    # Convert to torch tensors
    logits = torch.tensor(logits)
    labels = torch.tensor(labels)

    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()

    # CrossEntropy loss expects shape (batch_size * seq_len, vocab_size)
    # and labels shape (batch_size * seq_len)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(
        shift_logits.view(-1, shift_logits.size(-1)),
        shift_labels.view(-1)
    )

    # Convert to perplexity
    perplexity = torch.exp(loss)

    return {
        "eval_loss": loss.item(),
        "eval_perplexity": perplexity.item()
    }

## Step 9: Initializing the Trainer

In this step, we instantiate the HuggingFace `Trainer` class to orchestrate the fine-tuning process. We pass in:

- The **LoRA-augmented model**
- The **training arguments** (including learning rate, batch size, scheduler, logging, etc.)
- Our **tokenized dataset**, with training and validation splits
- An **early stopping callback** to terminate training if no improvement is seen for several epochs

This setup ensures modular, device-agnostic training with built-in support for checkpointing, evaluation, logging, and optimizer scheduling.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=6)]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Perplexity
1,1.375700,0.374699,1.454545
2,0.357300,0.304837,1.356403
3,0.276400,0.290156,1.336635
4,0.255100,0.286021,1.331119
5,0.242200,0.285445,1.330353


TrainOutput(global_step=78, training_loss=0.45214030681512296, metrics={'train_runtime': 320.6777, 'train_samples_per_second': 3.929, 'train_steps_per_second': 0.243, 'total_flos': 2.5755886798503936e+16, 'train_loss': 0.45214030681512296, 'epoch': 5.60377358490566})

## Step 10: Saving the Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model_path = "./models/finetuned-mistral"

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./models/finetuned-mistral/tokenizer_config.json',
 './models/finetuned-mistral/special_tokens_map.json',
 './models/finetuned-mistral/tokenizer.model',
 './models/finetuned-mistral/added_tokens.json',
 './models/finetuned-mistral/tokenizer.json')

## Step 11: Running Inference to Verify Behaviour

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)  # No device here!

instruction = "### Question:\nWhat is the benefit of using LoRA for fine-tuning large language models?\n\n### Answer:\n"
output = pipe(instruction, max_new_tokens=150, do_sample=True, temperature=0.7)

print(output[0]["generated_text"])

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJa

### Question:
What is the benefit of using LoRA for fine-tuning large language models?

### Answer:
LoRA enables selective and sparse fine-tuning, allowing updates to be focused on specific task-relevant dimensions, reducing computational requirements and memory footprint, especially for tasks with limited data. Additionally, it maintains a consistent behavioral profile, preserving model integrity.


In [1]:
pip install nbformat --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [2]:
import os
# List the notebook directory to confirm the file exists
os.listdir("/content/drive/MyDrive/llm-finetuning-project/llm-finetuning-summarizer/notebooks")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/llm-finetuning-project/llm-finetuning-summarizer/notebooks'

In [ ]:
import nbformat

notebook_path = "/content/drive/MyDrive/llm-finetuning-project/llm-finetuning-summarizer/notebooks/06_finetuning.ipynb"

with open(notebook_path, "r") as f:
    nb = nbformat.read(f, as_version=4)

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

with open(notebook_path, "w") as f:
    nbformat.write(nb, f)

print("Notebook fixed and saved successfully!")

FileNotFoundError: [Errno 2] No such file or directory